In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil
import os

In [2]:
PROJECT = 'kouzoh-p-michal'   
BUCKET = 'ml-training-kouzoh-p-michal-appspot-com' 
REGION = 'us-central1' 

In [3]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8' 

## ensure we're using python2 env
os.environ['CLOUDSDK_PYTHON'] = 'python2'

In [4]:
%%bash
gsutil ls gs://$BUCKET/taxifare/ch4/taxi_preproc/

gs://ml-training-kouzoh-p-michal-appspot-com/taxifare/ch4/taxi_preproc/train.csv-00000-of-00001
gs://ml-training-kouzoh-p-michal-appspot-com/taxifare/ch4/taxi_preproc/valid.csv-00000-of-00001
gs://ml-training-kouzoh-p-michal-appspot-com/taxifare/ch4/taxi_preproc/tmp/


In [6]:
# In CSV, label is the first column, after the features, followed by the key
CSV_COLUMNS = ['fare_amount', 'dayofweek', 'hourofday', 'pickuplon', 'pickuplat', 'dropofflon', 'dropofflat', 'passengers', 'key']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df_train = pd.read_csv('./data/train.csv', header = None, names = CSV_COLUMNS)
df_valid = pd.read_csv('./data/valid.csv', header = None, names = CSV_COLUMNS)

In [8]:
def make_train_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000
  )

def make_eval_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

def make_prediction_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = None,
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

def make_feature_cols():
  input_columns = [
      tf.feature_column.numeric_column('pickuplon'),
      tf.feature_column.numeric_column('pickuplat'),
      tf.feature_column.numeric_column('dropofflon'),
      tf.feature_column.numeric_column('dropofflat')
  ]
  return input_columns

In [9]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'data/taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.LinearRegressor(
      feature_columns = make_feature_cols(), model_dir = OUTDIR)

model.train(input_fn = make_train_input_fn(df_train, num_epochs = 10))
print "trained"

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3491608690>, '_model_dir': 'data/taxi_trained', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, 

INFO:tensorflow:global_step/sec: 212.82
INFO:tensorflow:loss = 7487.287, step = 6201 (0.474 sec)
INFO:tensorflow:global_step/sec: 219.245
INFO:tensorflow:loss = 6784.204, step = 6301 (0.458 sec)
INFO:tensorflow:global_step/sec: 191.328
INFO:tensorflow:loss = 5876.2734, step = 6401 (0.518 sec)
INFO:tensorflow:global_step/sec: 158.02
INFO:tensorflow:loss = 5681.354, step = 6501 (0.634 sec)
INFO:tensorflow:global_step/sec: 149.172
INFO:tensorflow:loss = 14781.336, step = 6601 (0.668 sec)
INFO:tensorflow:global_step/sec: 200.874
INFO:tensorflow:loss = 7839.33, step = 6701 (0.502 sec)
INFO:tensorflow:global_step/sec: 205.383
INFO:tensorflow:loss = 10606.764, step = 6801 (0.483 sec)
INFO:tensorflow:global_step/sec: 193.636
INFO:tensorflow:loss = 7041.8467, step = 6901 (0.523 sec)
INFO:tensorflow:global_step/sec: 213.274
INFO:tensorflow:loss = 7188.7925, step = 7001 (0.463 sec)
INFO:tensorflow:global_step/sec: 187.654
INFO:tensorflow:loss = 11654.693, step = 7101 (0.532 sec)
INFO:tensorflow:g

INFO:tensorflow:global_step/sec: 227.656
INFO:tensorflow:loss = 5210.664, step = 14501 (0.438 sec)
INFO:tensorflow:global_step/sec: 211.668
INFO:tensorflow:loss = 8638.676, step = 14601 (0.471 sec)
INFO:tensorflow:global_step/sec: 224.637
INFO:tensorflow:loss = 15177.693, step = 14701 (0.447 sec)
INFO:tensorflow:global_step/sec: 218.144
INFO:tensorflow:loss = 8270.1045, step = 14801 (0.456 sec)
INFO:tensorflow:global_step/sec: 207.04
INFO:tensorflow:loss = 8816.738, step = 14901 (0.486 sec)
INFO:tensorflow:global_step/sec: 152.181
INFO:tensorflow:loss = 13095.191, step = 15001 (0.656 sec)
INFO:tensorflow:global_step/sec: 111.207
INFO:tensorflow:loss = 10465.657, step = 15101 (0.899 sec)
INFO:tensorflow:global_step/sec: 218.513
INFO:tensorflow:loss = 11942.415, step = 15201 (0.456 sec)
INFO:tensorflow:global_step/sec: 215.901
INFO:tensorflow:loss = 7076.672, step = 15301 (0.467 sec)
INFO:tensorflow:global_step/sec: 217.309
INFO:tensorflow:loss = 17448.344, step = 15401 (0.457 sec)
INFO:

INFO:tensorflow:global_step/sec: 213.339
INFO:tensorflow:loss = 5441.7915, step = 22801 (0.469 sec)
INFO:tensorflow:global_step/sec: 218.047
INFO:tensorflow:loss = 13939.16, step = 22901 (0.457 sec)
INFO:tensorflow:global_step/sec: 214.119
INFO:tensorflow:loss = 9489.865, step = 23001 (0.468 sec)
INFO:tensorflow:global_step/sec: 216.685
INFO:tensorflow:loss = 8886.34, step = 23101 (0.462 sec)
INFO:tensorflow:global_step/sec: 219.537
INFO:tensorflow:loss = 10262.716, step = 23201 (0.452 sec)
INFO:tensorflow:global_step/sec: 188.255
INFO:tensorflow:loss = 10931.082, step = 23301 (0.548 sec)
INFO:tensorflow:global_step/sec: 165.77
INFO:tensorflow:loss = 9373.268, step = 23401 (0.586 sec)
INFO:tensorflow:global_step/sec: 216.299
INFO:tensorflow:loss = 9019.709, step = 23501 (0.464 sec)
INFO:tensorflow:global_step/sec: 226.338
INFO:tensorflow:loss = 9042.885, step = 23601 (0.441 sec)
INFO:tensorflow:global_step/sec: 175.734
INFO:tensorflow:loss = 7073.54, step = 23701 (0.572 sec)
INFO:tenso

INFO:tensorflow:loss = 6849.811, step = 31001 (0.496 sec)
INFO:tensorflow:global_step/sec: 139.123
INFO:tensorflow:loss = 13846.792, step = 31101 (0.717 sec)
INFO:tensorflow:global_step/sec: 166.979
INFO:tensorflow:loss = 7675.3438, step = 31201 (0.602 sec)
INFO:tensorflow:global_step/sec: 142.243
INFO:tensorflow:loss = 12204.156, step = 31301 (0.698 sec)
INFO:tensorflow:global_step/sec: 108.922
INFO:tensorflow:loss = 9116.072, step = 31401 (0.936 sec)
INFO:tensorflow:global_step/sec: 136.422
INFO:tensorflow:loss = 4637.672, step = 31501 (0.718 sec)
INFO:tensorflow:global_step/sec: 164.836
INFO:tensorflow:loss = 11025.847, step = 31601 (0.603 sec)
INFO:tensorflow:global_step/sec: 199.774
INFO:tensorflow:loss = 13683.27, step = 31701 (0.501 sec)
INFO:tensorflow:global_step/sec: 199.167
INFO:tensorflow:loss = 20254.223, step = 31801 (0.502 sec)
INFO:tensorflow:global_step/sec: 203.534
INFO:tensorflow:loss = 12136.992, step = 31901 (0.493 sec)
INFO:tensorflow:global_step/sec: 202.868
INFO

In [10]:
def print_rmse(model, df):
  metrics = model.evaluate(input_fn = make_eval_input_fn(df))
  print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))
print_rmse(model, df_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-14-15:48:17
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from data/taxi_trained/model.ckpt-33316
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-02-14-15:48:24
INFO:tensorflow:Saving dict for global step 33316: average_loss = 90.65271, global_step = 33316, label/mean = 11.263868, loss = 11599.479, prediction/mean = 10.001255
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 33316: data/taxi_trained/model.ckpt-33316
RMSE on dataset = 9.52117156982


In [11]:
predictions = model.predict(input_fn = make_prediction_input_fn(df_valid))
for items in predictions:
  print(items)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from data/taxi_trained/model.ckpt-33316
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([10.0028105], dtype=float32)}
{'predictions': array([9.998662], dtype=float32)}
{'predictions': array([9.999508], dtype=float32)}
{'predictions': array([10.000117], dtype=float32)}
{'predictions': array([10.002256], dtype=float32)}
{'predictions': array([9.99955], dtype=float32)}
{'predictions': array([9.995907], dtype=float32)}
{'predictions': array([9.992205], dtype=float32)}
{'predictions': array([10.000203], dtype=float32)}
{'predictions': array([10.000424], dtype=float32)}
{'predictions': array([10.005862], dtype=float32)}
{'predictions': array([10.002884], dtype=float32)}
{'predictions': array([9.995031], dtype=float32)}
{'predictions': array([10.00791], dtype=float32)}
{'predictions': array([

{'predictions': array([10.005334], dtype=float32)}
{'predictions': array([10.002647], dtype=float32)}
{'predictions': array([10.004328], dtype=float32)}
{'predictions': array([10.002273], dtype=float32)}
{'predictions': array([10.005237], dtype=float32)}
{'predictions': array([10.005146], dtype=float32)}
{'predictions': array([10.004302], dtype=float32)}
{'predictions': array([10.002326], dtype=float32)}
{'predictions': array([10.005614], dtype=float32)}
{'predictions': array([9.993914], dtype=float32)}
{'predictions': array([10.00325], dtype=float32)}
{'predictions': array([10.001515], dtype=float32)}
{'predictions': array([10.001613], dtype=float32)}
{'predictions': array([9.999971], dtype=float32)}
{'predictions': array([10.001933], dtype=float32)}
{'predictions': array([10.000072], dtype=float32)}
{'predictions': array([10.00124], dtype=float32)}
{'predictions': array([9.99921], dtype=float32)}
{'predictions': array([10.002252], dtype=float32)}
{'predictions': array([10.003807], dt

{'predictions': array([10.002634], dtype=float32)}
{'predictions': array([10.000013], dtype=float32)}
{'predictions': array([10.001496], dtype=float32)}
{'predictions': array([10.003768], dtype=float32)}
{'predictions': array([10.003187], dtype=float32)}
{'predictions': array([10.002361], dtype=float32)}
{'predictions': array([10.001088], dtype=float32)}
{'predictions': array([9.9993105], dtype=float32)}
{'predictions': array([10.00299], dtype=float32)}
{'predictions': array([10.002704], dtype=float32)}
{'predictions': array([10.002012], dtype=float32)}
{'predictions': array([10.005237], dtype=float32)}
{'predictions': array([9.999614], dtype=float32)}
{'predictions': array([9.999785], dtype=float32)}
{'predictions': array([10.002795], dtype=float32)}
{'predictions': array([10.0018635], dtype=float32)}
{'predictions': array([10.006084], dtype=float32)}
{'predictions': array([10.002049], dtype=float32)}
{'predictions': array([10.001947], dtype=float32)}
{'predictions': array([9.999165],

{'predictions': array([10.002002], dtype=float32)}
{'predictions': array([10.007137], dtype=float32)}
{'predictions': array([10.000879], dtype=float32)}
{'predictions': array([10.002169], dtype=float32)}
{'predictions': array([10.001121], dtype=float32)}
{'predictions': array([10.00011], dtype=float32)}
{'predictions': array([10.001511], dtype=float32)}
{'predictions': array([9.999842], dtype=float32)}
{'predictions': array([9.999938], dtype=float32)}
{'predictions': array([10.0013275], dtype=float32)}
{'predictions': array([9.998894], dtype=float32)}
{'predictions': array([9.986358], dtype=float32)}
{'predictions': array([10.005964], dtype=float32)}
{'predictions': array([10.00284], dtype=float32)}
{'predictions': array([10.002075], dtype=float32)}
{'predictions': array([9.99974], dtype=float32)}
{'predictions': array([10.003313], dtype=float32)}
{'predictions': array([10.0011015], dtype=float32)}
{'predictions': array([10.001211], dtype=float32)}
{'predictions': array([10.000502], dt

{'predictions': array([10.000846], dtype=float32)}
{'predictions': array([10.001023], dtype=float32)}
{'predictions': array([10.001163], dtype=float32)}
{'predictions': array([10.000524], dtype=float32)}
{'predictions': array([10.001708], dtype=float32)}
{'predictions': array([9.995004], dtype=float32)}
{'predictions': array([10.003639], dtype=float32)}
{'predictions': array([9.999132], dtype=float32)}
{'predictions': array([9.994269], dtype=float32)}
{'predictions': array([10.001961], dtype=float32)}
{'predictions': array([10.00116], dtype=float32)}
{'predictions': array([10.002984], dtype=float32)}
{'predictions': array([10.004151], dtype=float32)}
{'predictions': array([10.002246], dtype=float32)}
{'predictions': array([9.999616], dtype=float32)}
{'predictions': array([9.999409], dtype=float32)}
{'predictions': array([10.000832], dtype=float32)}
{'predictions': array([10.003734], dtype=float32)}
{'predictions': array([10.0031595], dtype=float32)}
{'predictions': array([10.000249], d

...

{'predictions': array([9.999975], dtype=float32)}
{'predictions': array([10.000508], dtype=float32)}
{'predictions': array([10.002591], dtype=float32)}
{'predictions': array([10.001569], dtype=float32)}
{'predictions': array([10.00059], dtype=float32)}
{'predictions': array([9.999578], dtype=float32)}
{'predictions': array([10.001893], dtype=float32)}
{'predictions': array([10.001163], dtype=float32)}
{'predictions': array([10.003255], dtype=float32)}
{'predictions': array([10.002484], dtype=float32)}
{'predictions': array([10.000525], dtype=float32)}
{'predictions': array([10.005094], dtype=float32)}
{'predictions': array([10.0017], dtype=float32)}
{'predictions': array([10.00276], dtype=float32)}
{'predictions': array([10.002397], dtype=float32)}
{'predictions': array([10.002245], dtype=float32)}
{'predictions': array([10.001147], dtype=float32)}
{'predictions': array([10.00189], dtype=float32)}
{'predictions': array([10.003657], dtype=float32)}
{'predictions': array([10.001295], dty

{'predictions': array([9.987217], dtype=float32)}
{'predictions': array([10.000508], dtype=float32)}
{'predictions': array([10.000951], dtype=float32)}
{'predictions': array([10.00475], dtype=float32)}
{'predictions': array([9.980765], dtype=float32)}
{'predictions': array([10.0025215], dtype=float32)}
{'predictions': array([10.00543], dtype=float32)}
{'predictions': array([10.00081], dtype=float32)}
{'predictions': array([10.0002365], dtype=float32)}
{'predictions': array([10.001469], dtype=float32)}
{'predictions': array([9.998591], dtype=float32)}
{'predictions': array([10.000174], dtype=float32)}
{'predictions': array([10.003484], dtype=float32)}
{'predictions': array([10.005994], dtype=float32)}
{'predictions': array([10.001841], dtype=float32)}
{'predictions': array([10.004246], dtype=float32)}
{'predictions': array([10.0011215], dtype=float32)}
{'predictions': array([10.003438], dtype=float32)}
{'predictions': array([10.002437], dtype=float32)}
{'predictions': array([9.998189], 

{'predictions': array([10.002805], dtype=float32)}
{'predictions': array([10.00324], dtype=float32)}
{'predictions': array([10.004781], dtype=float32)}
{'predictions': array([10.001351], dtype=float32)}
{'predictions': array([10.002476], dtype=float32)}
{'predictions': array([9.999225], dtype=float32)}
{'predictions': array([10.002167], dtype=float32)}
{'predictions': array([10.002248], dtype=float32)}
{'predictions': array([10.004119], dtype=float32)}
{'predictions': array([10.002299], dtype=float32)}
{'predictions': array([10.001755], dtype=float32)}
{'predictions': array([10.002683], dtype=float32)}
{'predictions': array([10.002918], dtype=float32)}
{'predictions': array([10.000503], dtype=float32)}
{'predictions': array([10.001997], dtype=float32)}
{'predictions': array([10.006112], dtype=float32)}
{'predictions': array([10.0020485], dtype=float32)}
{'predictions': array([10.001062], dtype=float32)}
{'predictions': array([10.002579], dtype=float32)}
{'predictions': array([10.002383

{'predictions': array([10.002425], dtype=float32)}
{'predictions': array([10.003394], dtype=float32)}
{'predictions': array([10.003207], dtype=float32)}
{'predictions': array([10.00277], dtype=float32)}
{'predictions': array([10.002156], dtype=float32)}
{'predictions': array([9.999322], dtype=float32)}
{'predictions': array([9.997677], dtype=float32)}
{'predictions': array([9.998944], dtype=float32)}
{'predictions': array([10.002087], dtype=float32)}
{'predictions': array([10.003605], dtype=float32)}
{'predictions': array([10.005256], dtype=float32)}
{'predictions': array([9.976696], dtype=float32)}
{'predictions': array([10.002944], dtype=float32)}
{'predictions': array([10.000932], dtype=float32)}
{'predictions': array([9.997523], dtype=float32)}
{'predictions': array([9.999159], dtype=float32)}
{'predictions': array([10.002214], dtype=float32)}
{'predictions': array([10.00281], dtype=float32)}
{'predictions': array([10.003234], dtype=float32)}
{'predictions': array([10.002103], dtyp

KeyboardInterrupt: 